In [ ]:
# Evan Collins
# IDCE 30274
# Created 5/6/2021
# Creating a map of planned unit development and zoing in DC
# # Based on a tutorial
# Prepare environment 
!pip install geopandas
!apt-get install -y libspatialindex-dev
!pip install rtree

import pandas as pd  # provides interface for interacting with tabular data
import geopandas as gpd  # combines the capabilities of pandas and shapely for geospatial operations
from shapely.geometry import Point, Polygon, MultiPolygon  # for manipulating text data into geospatial shapes
from shapely import wkt  # stands for "well known text," allows for interchange across GIS programs
import rtree  # supports geospatial join

from google.colab import drive
drive.mount('/content/gdrive')
root_path='gdrive/My Drive/GIS/'



In [ ]:
#read in PUD files
puds=gpd.read_file(root_path+'INPUT/Planned_Unit_Development_(PUDs).shp')
aff=pd.read_csv(root_path+'INPUT/Affordable_Housing.csv')
crosswalk=pd.read_csv(root_path+'INPUT/zoning_crosswalk.csv')

# Create a geometry column in the affordable housing dataframe
aff['geometry'] = aff.apply(lambda row: Point(row.X, row.Y), axis=1)
aff = gpd.GeoDataFrame(aff, crs={'init' :'epsg:4326'})

puds_aff=gpd.sjoin(puds, aff, op='intersects',how='left')

puds_info=puds_aff.merge(crosswalk[['Zone_Cat']],how='left',left_on='PUD_ZONING',right_on=crosswalk['Zone'])

print(f"Total count of PUDs: {puds_info.shape[0]}")
print(f"Count PUDs offering Affordable Housing: {puds_info.loc[~puds_info.PROJECT_NAME.isna()].shape[0]}")

# Create a map 
puds_info[puds_info['TOTAL_AFFORDABLE_UNITS']>0].plot(column='TOTAL_AFFORDABLE_UNITS',color='grey',figsize=(16,8))

# Export files
puds_info.to_file('puds_info.shp')
!cp puds_info.cpg 'gdrive/My Drive/GIS/OUTPUT'
!cp puds_info.dbf 'gdrive/My Drive/GIS/OUTPUT'
!cp puds_info.prj 'gdrive/My Drive/GIS/OUTPUT'
!cp puds_info.shp 'gdrive/My Drive/GIS/OUTPUT'
!cp puds_info.shx 'gdrive/My Drive/GIS/OUTPUT'